### Paquetes a instalar

In [1]:
#pip install opencv-python

### Importando librerias para procesamiento de imagenes

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.utils import to_categorical, plot_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Conv2D, MaxPooling2D, Activation, Flatten, Dropout, Dense
from tensorflow.keras import backend as K
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import numpy as np
import random
import cv2
import os
import glob

### PARÀMETROS GLOBALES

In [3]:
EPOCS=100
LR=1 #1e-3
BATCH_SIZE = 64
IMG_DIMS=(96,96,3)

In [4]:
data=[]
labels=[]

### Cargar imàgenes del dataset
#### Cargamos primero las imagenes de mujeres y su etiqueta de mujer
#### Cargamos las imagenes de hombres y su etiqueta de hombre

In [5]:
woman_image_files = [f for f in glob.glob('Gender/woman'+"/*",recursive=True) if not os.path.isdir(f)]
woman_image_files


['Gender/woman\\face_0.jpg',
 'Gender/woman\\face_1.jpg',
 'Gender/woman\\face_10.jpg',
 'Gender/woman\\face_100.jpg',
 'Gender/woman\\face_1000.jpg',
 'Gender/woman\\face_1001.jpg',
 'Gender/woman\\face_1002.jpg',
 'Gender/woman\\face_1003.jpg',
 'Gender/woman\\face_1004.jpg',
 'Gender/woman\\face_1005.jpg',
 'Gender/woman\\face_1006.jpg',
 'Gender/woman\\face_101.jpg',
 'Gender/woman\\face_1012.jpg',
 'Gender/woman\\face_1013.jpg',
 'Gender/woman\\face_1014.jpg',
 'Gender/woman\\face_1015.jpg',
 'Gender/woman\\face_1016.jpg',
 'Gender/woman\\face_1017.jpg',
 'Gender/woman\\face_1019.jpg',
 'Gender/woman\\face_102.jpg',
 'Gender/woman\\face_1020.jpg',
 'Gender/woman\\face_1021.jpg',
 'Gender/woman\\face_1022.jpg',
 'Gender/woman\\face_1023.jpg',
 'Gender/woman\\face_1024.jpg',
 'Gender/woman\\face_1025.jpg',
 'Gender/woman\\face_1026.jpg',
 'Gender/woman\\face_1027.jpg',
 'Gender/woman\\face_1028.jpg',
 'Gender/woman\\face_1029.jpg',
 'Gender/woman\\face_103.jpg',
 'Gender/woman\\face

In [6]:
man_image_files = [f for f in glob.glob('Gender/man'+"/*",recursive=True) if not os.path.isdir(f)]
man_image_files


['Gender/man\\face_0.jpg',
 'Gender/man\\face_1.jpg',
 'Gender/man\\face_10.jpg',
 'Gender/man\\face_1000.jpg',
 'Gender/man\\face_1001.jpg',
 'Gender/man\\face_1003.jpg',
 'Gender/man\\face_1004.jpg',
 'Gender/man\\face_1005.jpg',
 'Gender/man\\face_1006.jpg',
 'Gender/man\\face_1007.jpg',
 'Gender/man\\face_101.jpg',
 'Gender/man\\face_1010.jpg',
 'Gender/man\\face_1011.jpg',
 'Gender/man\\face_1012.jpg',
 'Gender/man\\face_1014.jpg',
 'Gender/man\\face_1016.jpg',
 'Gender/man\\face_1018.jpg',
 'Gender/man\\face_1019.jpg',
 'Gender/man\\face_1020.jpg',
 'Gender/man\\face_1021.jpg',
 'Gender/man\\face_1022.jpg',
 'Gender/man\\face_1023.jpg',
 'Gender/man\\face_1024.jpg',
 'Gender/man\\face_1025.jpg',
 'Gender/man\\face_1026.jpg',
 'Gender/man\\face_1028.jpg',
 'Gender/man\\face_1029.jpg',
 'Gender/man\\face_103.jpg',
 'Gender/man\\face_1030.jpg',
 'Gender/man\\face_1032.jpg',
 'Gender/man\\face_1033.jpg',
 'Gender/man\\face_1034.jpg',
 'Gender/man\\face_1035.jpg',
 'Gender/man\\face_1

In [7]:
def create_gender_data(image_files,labels,label,data):
    for img in image_files:
        image=cv2.imread(img)
    
        image = cv2.resize(image,(IMG_DIMS[0],IMG_DIMS[1]))
        image = img_to_array(image)
        data.append(image)
        labels.append(label)
        
        

In [8]:
create_gender_data(woman_image_files,labels,1,data) #woman
create_gender_data(man_image_files,labels,0,data) # man


In [9]:
data

[array([[[ 84., 122., 114.],
         [102., 136., 143.],
         [111., 144., 158.],
         ...,
         [110., 126., 162.],
         [ 99., 112., 147.],
         [ 93., 106., 142.]],
 
        [[102., 135., 144.],
         [107., 140., 151.],
         [116., 149., 167.],
         ...,
         [ 87.,  98., 131.],
         [102., 118., 155.],
         [103., 120., 157.]],
 
        [[103., 139., 138.],
         [115., 149., 163.],
         [118., 148., 175.],
         ...,
         [103., 115., 151.],
         [ 87.,  98., 126.],
         [ 93., 104., 139.]],
 
        ...,
 
        [[ 14.,  14.,  54.],
         [ 14.,  14.,  60.],
         [ 15.,  17.,  65.],
         ...,
         [ 91., 110., 175.],
         [ 48.,  65., 114.],
         [ 75.,  86., 145.]],
 
        [[ 11.,  14.,  52.],
         [ 14.,  16.,  57.],
         [ 16.,  17.,  61.],
         ...,
         [ 78.,  97., 153.],
         [ 66.,  81., 133.],
         [ 54.,  69., 125.]],
 
        [[ 11.,  14.,  52.],
 

### PRE PROCESAMIENTO

In [10]:
data = np.array(data,dtype="float")/255.0
#data = np.array(data)
data=data / 255.0
labels = np.array(labels)
data

array([[[[0.00129181, 0.0018762 , 0.00175317],
         [0.00156863, 0.0020915 , 0.00219915],
         [0.00170704, 0.00221453, 0.00242983],
         ...,
         [0.00169166, 0.00193772, 0.00249135],
         [0.00152249, 0.00172241, 0.00226067],
         [0.00143022, 0.00163014, 0.00218378]],

        [[0.00156863, 0.00207612, 0.00221453],
         [0.00164552, 0.00215302, 0.00232218],
         [0.00178393, 0.00229143, 0.00256824],
         ...,
         [0.00133795, 0.00150711, 0.00201461],
         [0.00156863, 0.00181469, 0.0023837 ],
         [0.00158401, 0.00184544, 0.00241446]],

        [[0.00158401, 0.00213764, 0.00212226],
         [0.00176855, 0.00229143, 0.00250673],
         [0.00181469, 0.00227605, 0.00269127],
         ...,
         [0.00158401, 0.00176855, 0.00232218],
         [0.00133795, 0.00150711, 0.00193772],
         [0.00143022, 0.00159938, 0.00213764]],

        ...,

        [[0.0002153 , 0.0002153 , 0.00083045],
         [0.0002153 , 0.0002153 , 0.00092272]

In [11]:
## Falta hacer shuffle a data sin perder los labels

### Dividir el dataset para el entrenamiento y la validaciòn

In [12]:
(x_train,x_test,y_train,y_test) = train_test_split(data,labels,test_size=0.2,random_state=42)
y_train = to_categorical(y_train,num_classes=2)
y_test = to_categorical(y_test,num_classes=2)

### Se aumenta el dataset

In [13]:
aug = ImageDataGenerator(rotation_range=25,width_shift_range=0.1,
                        height_shift_range=0.1,shear_range=0.2,zoom_range=0.2,
                        horizontal_flip=True, fill_mode="nearest")

## MODELADO

In [14]:
def buildModel(width,height,depth,classes):
    model= Sequential()
    inputShape = (height,width,depth)
    chanDim=-1
    
    if (K.image_data_format() == "channels_first"): #Returns a string, either 'channels_first' or 'channels_last'
        inputShape = (depth, height, width)
        chanDim = 1
    
    # The axis that should be normalized, after a Conv2D layer with data_format="channels_first", 
    # set axis=1 in BatchNormalization.

    model.add(Conv2D(32, (3,3), padding="same", input_shape=inputShape))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(3,3)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(64, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))

    model.add(Conv2D(128, (3,3), padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(axis=chanDim))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(1024))
    model.add(Activation("relu"))
    model.add(BatchNormalization())
    model.add(Dropout(0.5))

    model.add(Dense(classes))
    model.add(Activation("sigmoid"))

    return model


In [15]:
# build model
model = buildModel(width=IMG_DIMS[0], height=IMG_DIMS[1], depth=IMG_DIMS[2],classes=2)

# compile the model
opt = Adam(lr=LR, decay=LR/EPOCS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])



### Entrenar el modelo

In [16]:
x_train

array([[[[1.59938485e-03, 2.07612457e-03, 2.53748558e-03],
         [1.55324875e-03, 2.02998847e-03, 2.47597078e-03],
         [1.84544406e-03, 2.30680507e-03, 2.73740869e-03],
         ...,
         [2.10688197e-03, 2.47597078e-03, 2.93733180e-03],
         [2.18377547e-03, 2.53748558e-03, 3.01422530e-03],
         [2.06074587e-03, 2.41445598e-03, 2.92195309e-03]],

        [[1.19953864e-03, 1.69165705e-03, 2.13763937e-03],
         [1.16878124e-03, 1.64552095e-03, 2.09150327e-03],
         [1.41484045e-03, 1.87620146e-03, 2.32218378e-03],
         ...,
         [2.24529027e-03, 2.61437908e-03, 3.07574010e-03],
         [2.12226067e-03, 2.47597078e-03, 2.95271050e-03],
         [1.95309496e-03, 2.29142637e-03, 2.78354479e-03]],

        [[1.15340254e-03, 1.64552095e-03, 2.09150327e-03],
         [1.15340254e-03, 1.63014225e-03, 2.07612457e-03],
         [1.06113033e-03, 1.52249135e-03, 1.96847366e-03],
         ...,
         [2.06074587e-03, 2.42983468e-03, 2.89119569e-03],
         [

In [17]:
y_train

array([[0., 1.],
       [1., 0.],
       [0., 1.],
       ...,
       [0., 1.],
       [1., 0.],
       [0., 1.]], dtype=float32)

In [18]:
x_test

array([[[[4.76739715e-04, 4.61361015e-04, 4.92118416e-04],
         [5.22875817e-04, 5.53633218e-04, 5.69011918e-04],
         [3.53710111e-04, 4.15224913e-04, 4.30603614e-04],
         ...,
         [9.22722030e-05, 1.23029604e-04, 1.23029604e-04],
         [1.23029604e-04, 1.53787005e-04, 1.53787005e-04],
         [2.15301807e-04, 2.15301807e-04, 2.15301807e-04]],

        [[3.53710111e-04, 3.84467512e-04, 3.99846213e-04],
         [3.84467512e-04, 4.30603614e-04, 4.45982314e-04],
         [2.46059208e-04, 3.22952710e-04, 3.38331411e-04],
         ...,
         [1.53787005e-04, 1.53787005e-04, 1.53787005e-04],
         [1.38408304e-04, 1.53787005e-04, 1.53787005e-04],
         [1.23029604e-04, 1.53787005e-04, 1.53787005e-04]],

        [[2.92195309e-04, 3.22952710e-04, 3.38331411e-04],
         [2.76816609e-04, 3.38331411e-04, 3.53710111e-04],
         [1.99923106e-04, 2.76816609e-04, 2.92195309e-04],
         ...,
         [1.69165705e-04, 1.69165705e-04, 1.69165705e-04],
         [

In [19]:
y_test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [1., 0.],
       [0., 1.],
       [0., 1.],
       [0., 1.],
       [1., 0.],
       [1., 0.],
       [1., 0.],
       [1., 0.

In [ ]:
# train the model
#history = model.fit_generator(aug.flow(x_train, y_train, batch_size=BATCH_SIZE),
 #                       validation_data=(x_test,y_test),
  #                      steps_per_epoch=len(x_train),
   #                     epochs=EPOCS, verbose=1)

history=model.fit(x_train,y_train,epochs=EPOCS,batch_size=BATCH_SIZE)
    

Epoch 1/100
29/29 [==============================] - 43s 1s/step - loss: 44.1544 - accuracy: 0.6672
Epoch 2/100
29/29 [==============================] - 43s 1s/step - loss: 29.3664 - accuracy: 0.7079
Epoch 3/100
29/29 [==============================] - 41s 1s/step - loss: 20.9228 - accuracy: 0.7442
Epoch 4/100
29/29 [==============================] - 42s 1s/step - loss: 17.6191 - accuracy: 0.7279
Epoch 5/100
29/29 [==============================] - 42s 1s/step - loss: 16.7455 - accuracy: 0.7203
Epoch 6/100
29/29 [==============================] - 40s 1s/step - loss: 11.1693 - accuracy: 0.7821
Epoch 7/100
29/29 [==============================] - 43s 1s/step - loss: 10.6596 - accuracy: 0.7599
Epoch 8/100
29/29 [==============================] - 41s 1s/step - loss: 9.5720 - accuracy: 0.7854
Epoch 9/100
29/29 [==============================] - 40s 1s/step - loss: 3.6326 - accuracy: 0.7951
Epoch 10/100
29/29 [==============================] - 48s 2s/step - loss: 2.3145 - accuracy: 0.7892
Ep

### Visualizar el historico

In [ ]:
# plot training/validation loss/accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0,N), history.history["loss"], label="train_loss")
plt.plot(np.arange(0,N), history.history["val_loss"], label="val_loss")
plt.plot(np.arange(0,N), history.history["acc"], label="train_acc")
plt.plot(np.arange(0,N), history.history["val_acc"], label="val_acc")

plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper right")

### Salvar el modelo

In [ ]:

# save the model to disk
model.save('gender_detection.model')


# save plot to disk
plt.savefig('plot.png')

###### Review following video:
https://www.youtube.com/watch?v=WOuAI5DhHyU